# Mortality Cleaning cont'd 3- 
- for blank spaces to 99 
- Add ID columns for SQL



In [ ]:
# Essentials
import sys
import pandas as pd
pd.set_option('display.max_columns', 100)
import csv
import numpy as np
import os

# Plots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Stats
from scipy import stats
from scipy.stats import skew


In [ ]:
%%time 
# Load all files
# m2018 = pd.read_csv('data/rawdata/mort_2018.csv', low_memory=False)
m2017 = pd.read_csv('data/rawdata/mort_2017.csv', low_memory=False)
# m2016 = pd.read_csv('data/rawdata/mort_2016.csv', low_memory=False)
# m2015 = pd.read_csv('data/rawdata/mort_2015.csv', low_memory=False)
# m2014 = pd.read_csv('data/rawdata/mort_2014.csv', low_memory=False)
# m2013 = pd.read_csv('data/rawdata/mort_2013.csv', low_memory=False)
# m2012 = pd.read_csv('data/rawdata/mort_2012.csv', low_memory=False)
# m2011 = pd.read_csv('data/rawdata/mort_2011.csv', low_memory=False)

In [40]:
m2018.sample(7)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
1711980,1711981,8,2,41,,4,1,10,M,1,037 331306,1037,,33,13,6,,7,D,3,2018,N,1,C,U,9,99,V892397 114 38,V892,397,114,,38,0211T149 12V892 ...,2,11T149 12V892 ...,11T149,12V892,,,,,,,,,,,,,,,,,,,2,V892 T149 ...,V892,T149,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1057628,1057629,8,1,51,,5,1,2,F,1,059 371708,1059,,37,17,8,,4,M,7,2018,U,7,C,N,99,99,K703298 094 30,K703,298,94,,30,0511R628 21K703 61R18 62R568 63E039 ...,5,11R628 21K703 61R18 62R568 63E039 ...,11R628,21K703,61R18,62R568,63E039,,,,,,,,,,,,,,,,5,K703 E039 R18 R568 R628 ...,K703,E039,R18,R568,R628,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1258417,1258418,8,2,31,,3,1,3,F,1,084 422210,1084,,42,22,10,,1,M,6,2018,U,7,O,N,99,99,F03 175 111 37,F03,175,111,,37,0211J189 61F03 ...,2,11J189 61F03 ...,11J189,61F03,,,,,,,,,,,,,,,,,,,2,F03 J189 ...,F03,J189,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
880254,880255,8,1,31,,3,1,4,M,1,063 381808,1063,,38,18,8,,1,M,2,2018,U,7,C,N,99,99,G122194 111 37,G122,194,111,,37,0311G122 61J189 62F179 ...,3,11G122 61J189 62F179 ...,11G122,61J189,62F179,,,,,,,,,,,,,,,,,,3,G122 F179 J189 ...,G122,F179,J189,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2495541,2495542,8,1,31,,3,1,9,F,1,080 422210,1080,,42,22,10,,6,W,1,2018,U,7,C,N,99,99,N390330 111 37,N390,330,111,,37,0511N185 61D649 62G729 63I10 64N390 ...,5,11N185 61D649 62G729 63I10 64N390 ...,11N185,61D649,62G729,63I10,64N390,,,,,,,,,,,,,,,,4,N390 D649 G729 I120 ...,N390,D649,G729,I120,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2182044,2182045,8,1,31,,3,1,8,F,1,064 381808,1064,,38,18,8,,6,S,6,2018,U,7,C,U,99,99,J439266 084 28,J439,266,84,,28,0211R092 21J439 ...,2,11R092 21J439 ...,11R092,21J439,,,,,,,,,,,,,,,,,,,2,J439 R092 ...,J439,R092,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
465077,465078,8,1,21,,2,1,3,F,1,072 402009,1072,,40,20,9,,4,D,6,2018,U,0,B,N,99,99,C170079 043 15,C170,79,43,,15,0111C170 ...,1,11C170 ...,11C170,,,,,,,,,,,,,,,,,,,,1,C170 ...,C170,,,,,,,,,,,,,,,,,,,,01 211,1,99,2,1,1,260,2


In [41]:
m2017.sample(7)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
1788129,1788130,7,2,61,,6,1,4,M,1,048 351507,1048,,35,15,7,,1,D,5,2017,U,7,D,U,99,99,I255215 063 21,I255,215,63,,21,0111I255 ...,1,11I255 ...,11I255,,,,,,,,,,,,,,,,,,,,1,I255 ...,I255,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2269132,2269133,7,1,31,,3,1,2,M,1,073 402009,1073,,40,20,9,,3,D,3,2017,U,7,B,N,99,99,I219211 059 21,I219,211,59,,21,0311I490 21I219 31I251 ...,3,11I490 21I219 31I251 ...,11I490,21I219,31I251,,,,,,,,,,,,,,,,,,3,I219 I251 I490 ...,I219,I251,I490,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1320734,1320735,7,1,31,,3,1,7,F,1,093 442411,1093,,44,24,11,,5,W,6,2017,U,7,B,N,99,99,G309189 052 17,G309,189,52,,17,0111G309 ...,1,11G309 ...,11G309,,,,,,,,,,,,,,,,,,,,1,G309 ...,G309,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
903011,903012,7,1,61,,6,1,7,F,1,088 432311,1088,,43,23,11,,6,W,2,2017,U,7,B,N,99,99,I48 228 068 22,I48,228,68,,22,0411I469 21I500 31I48 32I10 ...,4,11I469 21I500 31I48 32I10 ...,11I469,21I500,31I48,32I10,,,,,,,,,,,,,,,,,4,I48 I10 I469 I500 ...,I48,I10,I469,I500,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
48505,48506,7,1,31,,3,1,11,M,1,078 412110,1078,,41,21,10,,4,W,4,2017,U,7,C,N,99,99,C189081 023 06,C189,81,23,,6,0111C189 ...,1,11C189 ...,11C189,,,,,,,,,,,,,,,,,,,,1,C189 ...,C189,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
470324,470325,7,1,51,,5,1,7,M,1,067 391909,1067,,39,19,9,,4,M,5,2017,U,7,C,Y,99,99,I250214 062 21,I250,214,62,,21,0211I119 12I250 ...,2,11I119 12I250 ...,11I119,12I250,,,,,,,,,,,,,,,,,,,2,I250 I119 ...,I250,I119,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2084943,2084944,7,1,31,,3,1,4,F,1,043 341406,1043,,34,14,6,,4,M,7,2017,N,1,C,Y,9,8,X42 420 122 39,X42,420,122,,39,0411T404 12X42 13T405 61T509 ...,4,11T404 12X42 13T405 61T509 ...,11T404,12X42,13T405,61T509,,,,,,,,,,,,,,,,,4,X42 T404 T405 T509 ...,X42,T404,T405,T509,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [6]:
m2011.head()

,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
0,2,12 0,12,,0,1,F,1,065 391909,1065,,39,19,9,,1,M,2,2011,U,7,U,N,,,C859129 039 13,C859,129,39,,13,0111C859 ...,1,11C859 ...,11C859,,,,,,,,,,,,,,,,,,,,1,C859 ...,C859,,,,,,,,,,,,,,,,,,,,01 11,1,,,1,1,100,6
1,1,00 0,0,,0,1,F,1,057 371708,1057,,37,17,8,,1,S,7,2011,U,7,U,N,,,C509104 029 09,C509,104,29,,9,0211R092 21C509 ...,2,11R092 21C509 ...,11R092,21C509,,,,,,,,,,,,,,,,,,,2,C509 R092 ...,C509,R092,,,,,,,,,,,,,,,,,,,03 23,3,,,2,3,100,8
2,1,14 0,14,,0,1,M,1,075 412110,1075,,41,21,10,,1,M,3,2011,U,7,U,N,,,I251215 063 21,I251,215,63,,21,0511K559 21I251 31I739 41J449 61N185 ...,5,11K559 21I251 31I739 41J449 61N185 ...,11K559,21I251,31I739,41J449,61N185,,,,,,,,,,,,,,,,5,I251 I739 J449 K559 N185 ...,I251,I739,J449,K559,N185,,,,,,,,,,,,,,,,01 11,1,,,1,1,100,6
3,1,16 0,16,,0,1,F,1,061 381808,1061,,38,18,8,,4,M,1,2011,U,7,U,N,,,I250214 062 21,I250,214,62,,21,0311I250 61F102 62J449 ...,3,11I250 61F102 62J449 ...,11I250,61F102,62J449,,,,,,,,,,,,,,,,,,3,I250 F102 J449 ...,I250,F102,J449,,,,,,,,,,,,,,,,,,01 11,1,,,1,1,100,6
4,1,12 0,12,,0,1,M,1,020 301004,1020,,30,10,4,,7,S,7,2011,N,1,U,N,9,,V869396 114 38,V869,396,114,,38,0211S099 12V869 ...,2,11S099 12V869 ...,11S099,12V869,,,,,,,,,,,,,,,,,,,2,V869 S099 ...,V869,S099,,,,,,,,,,,,,,,,,,,03 23,3,,,2,3,100,8


In [24]:
%%time
# replace blanks in activity_code with 99, change dtype to int64
# m2018['activity_code'] = m2018['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2018 activity_code complete')
m2017['activity_code'] = m2017['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2017 activity_code complete')
# m2016['activity_code'] = m2016['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2016 activity_code complete')
# m2015['activity_code'] = m2015['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2015 activity_code complete')
# m2014['activity_code'] = m2014['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2014 activity_code complete')
# m2013['activity_code'] = m2013['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2013 activity_code complete')
# m2012['activity_code'] = m2012['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2012 activity_code complete')
# m2011['activity_code'] = m2011['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2011 activity_code complete')

Wall time: 1min 7s


In [8]:
m2011['place_of_injury'].dtype

dtype('O')

In [25]:
%%time
# replace blanks in place_of_injury with 99, change dtype to int64
# m2018['place_of_injury'] = m2018['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2018 place_of_injury complete')
m2017['place_of_injury'] = m2017['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
print('m2017 place_of_injury complete')
# m2016['place_of_injury'] = m2016['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2016 place_of_injury complete')
# m2015['place_of_injury'] = m2015['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2015 place_of_injury complete')
# m2014['place_of_injury'] = m2014['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2014 place_of_injury complete')
# m2013['place_of_injury'] = m2013['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2013 place_of_injury complete')
# m2012['place_of_injury'] = m2012['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2012 place_of_injury complete')
# m2011['place_of_injury'] = m2011['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2011 place_of_injury complete')

m2017 place_of_injury complete
Wall time: 1min 12s


In [26]:
%%time
# replace blanks in bridged_race_flag, race_imputation_flag with 99, change dtype to int64
# m2018[['bridged_race_flag', 'race_imputation_flag']] = m2018[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2018 bridged_race_flag, race_imputation_flag complete')

%time m2017[['bridged_race_flag', 'race_imputation_flag']] = m2017[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
print('m2017 bridged_race_flag, race_imputation_flag complete')
# m2016[['bridged_race_flag', 'race_imputation_flag']] = m2016[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2016 bridged_race_flag, race_imputation_flag complete')

# %time m2015[['bridged_race_flag', 'race_imputation_flag']] = m2015[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2015 bridged_race_flag, race_imputation_flag complete')
# m2014[['bridged_race_flag', 'race_imputation_flag']] = m2014[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2014 bridged_race_flag, race_imputation_flag complete')

# %time m2013[['bridged_race_flag', 'race_imputation_flag']] = m2013[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2013 bridged_race_flag, race_imputation_flag complete')
# m2012[['bridged_race_flag', 'race_imputation_flag']] = m2012[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2012 bridged_race_flag, race_imputation_flag complete')

# %time m2011[['bridged_race_flag', 'race_imputation_flag']] = m2011[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2011 bridged_race_flag, race_imputation_flag complete')

Wall time: 10.7 s
m2017 bridged_race_flag, race_imputation_flag complete
Wall time: 10.7 s


In [263]:
# dfs = list('m2011', 'm2012', 'm2013', 'm2014', 'm2015', 'm2016', 'm2017', 'm2018')
# for df in dfs:
#     if (df['activity_code'].dtype == dtype('O')):
#         df['activity_code'] = df['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(0).astype('int64')
#         print(df['activity_code'].dtype)
#     else:
#         continue


In [27]:
# df.drop('column_name', axis=1, inplace=True)
m2017.drop('year_id', axis=1, inplace=True)
m2017.head()

,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
0,1,41,,4,1,1,M,1,071 402009,1071,,40,20,9,,1,M,2,2017,U,7,C,N,99,99,G20 188 051 37,G20,188,51,,37,0111G20 ...,1,11G20 ...,11G20,,,,,,,,,,,,,,,,,,,,1,G20 ...,G20,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1,1,11,,1,1,1,M,1,074 402009,1074,,40,20,9,,1,D,4,2017,U,7,C,N,99,99,M726315 111 37,M726,315,111,,37,0211A419 21M726 ...,2,11A419 21M726 ...,11A419,21M726,,,,,,,,,,,,,,,,,,,2,M726 A419 ...,M726,A419,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2,1,41,,4,1,1,M,1,059 371708,1059,,37,17,8,,4,M,3,2017,U,7,C,N,99,99,C159077 021 15,C159,77,21,,15,0211C159 61F179 ...,2,11C159 61F179 ...,11C159,61F179,,,,,,,,,,,,,,,,,,,2,C159 F179 ...,C159,F179,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
3,1,11,,1,1,1,M,1,090 442411,1090,,44,24,11,,4,M,3,2017,U,7,B,N,99,99,C61 113 033 11,C61,113,33,,11,0111C61 ...,1,11C61 ...,11C61,,,,,,,,,,,,,,,,,,,,1,C61 ...,C61,,,,,,,,,,,,,,,,,,,,03 23,3,99,99,2,3,100,8
4,1,51,,5,1,1,M,1,066 391909,1066,,39,19,9,,4,M,3,2017,U,7,C,N,99,99,J449267 086 28,J449,267,86,,28,0311J449 61C900 62F179 ...,3,11J449 61C900 62F179 ...,11J449,61C900,62F179,,,,,,,,,,,,,,,,,,3,J449 C900 F179 ...,J449,C900,F179,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [28]:
m2018.sample(3)

,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
1847412,8,1,11,,1,1,10,M,1,097 452511,1097,,45,25,11,,6,W,1,2018,U,7,E,N,99,99,F03 175 111 37,F03,175,111,,37,0211F03 21N189 ...,2,11F03 21N189 ...,11F03,21N189,,,,,,,,,,,,,,,,,,,2,F03 N189 ...,F03,N189,,,,,,,,,,,,,,,,,,,04 24,4,99,99,2,4,100,8
998084,8,1,31,,3,1,8,M,1,053 361607,1053,,36,16,7,,4,M,1,2018,U,7,C,N,99,99,C64 117 034 12,C64,117,34,,12,0111C64 ...,1,11C64 ...,11C64,,,,,,,,,,,,,,,,,,,,1,C64 ...,C64,,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7
926159,8,1,11,,1,1,9,M,1,080 422210,1080,,42,22,10,,4,M,1,2018,U,7,B,N,99,99,C833129 039 13,C833,129,39,,13,0111C833 ...,1,11C833 ...,11C833,,,,,,,,,,,,,,,,,,,,1,C833 ...,C833,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,235,4


In [29]:
%%time
m2018.insert(0, 'year_id', '8')
m2017.insert(0, 'year_id', '7')
m2016.insert(0, 'year_id', '6')
m2015.insert(0, 'year_id', '5')
m2014.insert(0, 'year_id', '4')
m2013.insert(0, 'year_id', '3')
m2012.insert(0, 'year_id', '2')
m2011.insert(0, 'year_id', '1')

In [30]:
%%time
m2018.insert(0, 'person_id', np.arange(1,len(m2018)+1,1))
m2017.insert(0, 'person_id', np.arange(1,len(m2017)+1,1))
m2016.insert(0, 'person_id', np.arange(1,len(m2016)+1,1))
m2015.insert(0, 'person_id', np.arange(1,len(m2015)+1,1))
m2014.insert(0, 'person_id', np.arange(1,len(m2014)+1,1))
m2013.insert(0, 'person_id', np.arange(1,len(m2013)+1,1))
m2012.insert(0, 'person_id', np.arange(1,len(m2012)+1,1))
m2011.insert(0, 'person_id', np.arange(1,len(m2011)+1,1))

Wall time: 154 ms


In [35]:
m2011.sample(5)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
1232037,1232038,1,1,41,,4,1,8,M,1,082 422210,1082,,42,22,10,,4,M,7,2011,U,7,B,N,99,99,C349093 027 08,C349,93,27,,8,0411C349 21J449 61F101 62F179 ...,4,11C349 21J449 61F101 62F179 ...,11C349,21J449,61F101,62F179,,,,,,,,,,,,,,,,,4,C349 F101 F179 J449 ...,C349,F101,F179,J449,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1201508,1201509,1,1,31,,3,1,9,M,1,067 391909,1067,,39,19,9,,6,M,1,2011,U,7,C,N,99,99,J449267 086 28,J449,267,86,,28,0411J449 21I500 31I10 61F179 ...,4,11J449 21I500 31I10 61F179 ...,11J449,21I500,31I10,61F179,,,,,,,,,,,,,,,,,4,J449 F179 I10 I500 ...,J449,F179,I10,I500,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1894354,1894355,1,1,15 0,15,,0,6,F,1,076 412110,1076,,41,21,10,,1,W,1,2011,U,7,U,U,99,99,K729301 111 37,K729,301,111,,37,0211N179 21K729 ...,2,11N179 21K729 ...,11N179,21K729,,,,,,,,,,,,,,,,,,,2,K729 N179 ...,K729,N179,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7
2244441,2244442,1,2,61,,6,1,12,F,1,070 402009,1070,,40,20,9,,1,M,6,2011,U,7,C,N,99,99,C56 109 032 10,C56,109,32,,10,0211C56 12C787 ...,2,11C56 12C787 ...,11C56,12C787,,,,,,,,,,,,,,,,,,,2,C56 C787 ...,C56,C787,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
259050,259051,1,1,31,,3,1,7,F,1,094 442411,1094,,44,24,11,,4,W,5,2011,U,0,B,N,99,99,F03 175 111 37,F03,175,111,,37,0111F03 ...,1,11F03 ...,11F03,,,,,,,,,,,,,,,,,,,,1,F03 ...,F03,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [44]:
m2011['edu_2003_rev'].dtype

dtype('O')

In [45]:
m2011.groupby('edu_2003_rev').size()

edu_2003_rev
     674312
1    233153
2    204862
3    765032
4    224101
5     97110
6    174626
7     66149
8     29473
9     46640
dtype: int64

In [46]:
edu_1989_dtl=pd.read_excel('./keys/edu_1989_rev_LK.xlsx' )
edu_2003_dtl=pd.read_excel('./keys/edu_2003_rev_LK.xlsx' )

In [47]:
edu_1989_dtl.sample(3)

,edu_1989_rev,edu_1989_Desc
16,16,BA BS
7,7,1-8 Grd
12,12,HsGedCom


In [52]:
edu_1989_dtl['edu_1989_Desc'].apply(len)

0     8
1     7
2     7
3     7
4     7
5     7
6     7
7     7
8     7
9     7
10    7
11    7
12    8
13    7
14    7
15    7
16    5
17    6
18    6
Name: edu_1989_Desc, dtype: int64

In [81]:
edu_2003_dtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   edu_2003_rev   9 non-null      int64 
 1   edu_2003_Desc  9 non-null      object
dtypes: int64(1), object(1)
memory usage: 272.0+ bytes


In [75]:
m2017['edu_2003_rev'].dtype

dtype('O')

In [85]:
edu_2003_dtl['edu_2003_rev'].dtype

dtype('int64')

In [53]:
edu_2003_dtl['edu_2003_Desc'].apply(len)

0    7
1    7
2    8
3    8
4    5
5    5
6    7
7    8
8    6
Name: edu_2003_Desc, dtype: int64

In [54]:
edu_1989_dtl["edu_1989_rev"] = edu_1989_dtl["edu_1989_rev"].apply(str)
edu_1989_dtl["edu_1989_rev"] = edu_1989_dtl["edu_1989_rev"].str.zfill(2)

In [67]:
%%time
# m2018_t = pd.merge(m2018, edu_1989_dtl,on='edu_1989_rev',how='left')
m2017_t = pd.merge(m2017, edu_1989_dtl,on='edu_1989_rev',how='left')
# m2016_t = pd.merge(m2016, edu_1989_dtl,on='edu_1989_rev',how='left')

In [68]:
m2017_t.sample(3)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode,edu_1989_Desc
1449074,1449075,7,1,21,,2,1,4,F,1,090 442411,1090,,44,24,11,,4,W,4,2017,U,7,B,N,99,99,F03 175 111 37,F03,175,111,,37,0211E46 61F03 ...,2,11E46 61F03 ...,11E46,61F03,,,,,,,,,,,,,,,,,,,2,F03 E46 ...,F03,E46,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7,NaN
921660,921661,7,2,31,,3,1,9,F,1,082 422210,1082,,42,22,10,,1,W,7,2017,U,7,C,N,99,99,I469228 068 22,I469,228,68,,22,0211J960 21I469 ...,2,11J960 21I469 ...,11J960,21I469,,,,,,,,,,,,,,,,,,,2,I469 J960 ...,I469,J960,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6,NaN
2547058,2547059,7,3,31,,3,1,11,F,1,061 381808,1061,,38,18,8,,1,M,6,2017,U,7,C,N,99,99,I350225 068 22,I350,225,68,,22,0511I469 21I350 31T828 41Y831 42I359 ...,5,11I469 21I350 31T828 41Y831 42I359 ...,11I469,21I350,31T828,41Y831,42I359,,,,,,,,,,,,,,,,4,I350 I469 T828 Y831 ...,I350,I469,T828,Y831,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6,NaN


In [94]:
edu_2003_dtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   edu_2003_rev   9 non-null      object
 1   edu_2003_Desc  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [93]:
# m2017_t['edu_2003_rev'].to_numeric

edu_2003_dtl['edu_2003_rev'] = edu_2003_dtl['edu_2003_rev'].apply(str)

In [95]:
%%time
# m2018_t = pd.merge(m2018_t, edu_2003_dtl,on='edu_2003_rev',how='left')
# m2017_t = pd.merge(m2017_t, edu_2003_dtl,on='edu_2003_rev',how='left')
m2017_t = pd.merge(m2017_t, edu_2003_dtl,on='edu_2003_rev',how='left')

Wall time: 1min 32s


In [100]:
m2017.sample(3)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_1989_Desc,edu_2003_rev,edu_2003_Desc,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
2106644,2106645,7,2,31,,NaN,3,HsGedCom,1,6,F,1,072 402009,1072,,40,20,9,,1,D,5,2017,U,7,C,N,9,9,W80 410 123 39,W80,410,123,,39,0411J969 21T179 22W80 61F179 ...,4,11J969 21T179 22W80 61F179 ...,11J969,21T179,22W80,61F179,,,,,,,,,,,,,,,,,4,W80 F179 J969 T179 ...,W80,F179,J969,T179,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
266043,266044,7,1,41,,NaN,4,SomeCllg,1,6,M,1,066 391909,1066,,39,19,9,,4,M,7,2017,U,0,C,N,99,99,I429227 068 22,I429,227,68,,22,0611I469 21I500 31I429 61E119 62N189 63I499 ...,6,11I469 21I500 31I429 61E119 62N189 63I499 ...,11I469,21I500,31I429,61E119,62N189,63I499,,,,,,,,,,,,,,,5,I429 E112 I469 I499 I500 ...,I429,E112,I469,I499,I500,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2048325,2048326,7,1,61,,NaN,6,BA BS,1,11,F,1,088 432311,1088,,43,23,11,,6,D,7,2017,U,7,C,N,99,99,F03 175 111 37,F03,175,111,,37,0311R688 21R628 31F03 ...,3,11R688 21R628 31F03 ...,11R688,21R628,31F03,,,,,,,,,,,,,,,,,,3,F03 R628 R688 ...,F03,R628,R688,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [102]:
m2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2813503 entries, 0 to 2813502
Data columns (total 88 columns):
 #   Column                           Dtype 
---  ------                           ----- 
 0   person_id                        int32 
 1   year_id                          object
 2   resident_status                  int64 
 3   edu                              object
 4   edu_1989_rev                     object
 5   edu_1989_Desc                    object
 6   edu_2003_rev                     object
 7   edu_2003_Desc                    object
 8   edu_reporting_flag               int64 
 9   month_of_death                   int64 
 10  sex                              object
 11  reported_age_tmstmp_flag         int64 
 12  reported_age                     object
 13  age_detail                       int64 
 14  age_sub_flag                     object
 15  age_recode_52                    int64 
 16  age_recode_27                    int64 
 17  age_recode_12              

In [97]:
# m2018.insert(0, 'year_id', '8')
# m2018.insert(5, 'edu_1989_Desc', m2018_t['edu_1989_Desc'])
m2017.insert(5, 'edu_1989_Desc', m2017_t['edu_1989_Desc'])


In [98]:
# m2018.insert(7, 'edu_2003_Desc', m2018_t['edu_2003_Desc'])
m2017.insert(7, 'edu_2003_Desc', m2017_t['edu_2003_Desc'])

In [101]:
%%time
# m2018.to_csv('data/rawdata/mort_2018.csv', index = False, header = True)
# print('m2018 file complete')
m2017.to_csv('data/rawdata/mort_2017.csv', index = False, header = True)
print('m2017 file complete')
# m2016.to_csv('data/rawdata/mort_2016.csv', index = False, header = True)
# print('m2016 file complete')
# m2015.to_csv('data/rawdata/mort_2015.csv', index = False, header = True)
# print('m2015 file complete')
# m2014.to_csv('data/rawdata/mort_2014.csv', index = False, header = True)
# print('m2014 file complete')
# m2013.to_csv('data/rawdata/mort_2013.csv', index = False, header = True)
# print('m2013 file complete')
# m2012.to_csv('data/rawdata/mort_2012.csv', index = False, header = True)
# print('m2012 file complete')
# m2011.to_csv('data/rawdata/mort_2011.csv', index = False, header = True)
# print('m2011 file complete')

m2017 file complete
Wall time: 3min 6s
